<a href="https://colab.research.google.com/github/Kekon130/ML_Practica2/blob/main/practica2_continuo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
# Importación de librerías
%pip install xgboost
import numpy as np
import pandas as pd
import sklearn.compose as Compose
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from xgboost import XGBClassifier

from sklearn.metrics import f1_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [250]:
train_url = "https://raw.githubusercontent.com/Kekon130/ML_Practica2/main/train.csv"
df_train = pd.read_csv(train_url)

test_url = "https://raw.githubusercontent.com/Kekon130/ML_Practica2/main/test.csv"
df_test = pd.read_csv(test_url)

df_test


submission = df_test['Customer_ID'].copy()


#pd.set_option('display.max_columns', None)
df_train

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,43161,3985,4,Lawrencez,40.0,856431180.0,Accountant,97384.44,8167.370000,1.0,...,Good,405.45,32.643281,336.0,No,77.284943,53.714219,High_spent_Large_value_payments,847.172909,Standard
1,40663,49494,2,Mohammedr,39.0,417453942.0,Writer,31369.58,2607.131667,8.0,...,Bad,4259.91,39.375507,144.0,Yes,171.748418,44.990811,High_spent_Large_value_payments,280.855383,Standard
2,111059,2925,6,Alistairn,17.0,499528888.0,Writer,30171.86,2772.321667,6.0,...,Bad,3406.51,34.039021,123.0,Yes,205.403926,40.781489,High_spent_Large_value_payments,247.322615,Standard
3,72139,21358,2,Dinesh Naire,22.0,859449371.0,Doctor,13100.02,1246.668333,7.0,...,Standard,727.29,34.747262,129.0,Yes,41.177132,23.005463,Low_spent_Large_value_payments,298.156832,Standard
4,119841,21221,4,Bakerg,26.0,485239890.0,Doctor,62313.54,5120.795000,6.0,...,Standard,817.64,24.545394,122.0,NM,196.679642,63.346750,Low_spent_Medium_value_payments,152.669902,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,42636,39869,7,Hendrik Sackmannb,50.0,217529489.0,Media_Manager,107761.29,9047.107500,4.0,...,Standard,34.09,41.899992,211.0,Yes,0.000000,150.220682,High_spent_Large_value_payments,984.123010,Standard
79996,114031,38305,2,Alexj,33.0,98861902.0,Scientist,16218.32,1098.526667,6.0,...,Bad,4466.33,24.275710,98.0,NM,64.533114,34.457893,High_spent_Small_value_payments,260.523798,Poor
79997,85991,9976,6,Luciana Lopezi,25.0,745835485.0,Engineer,29374.77,2377.897500,3.0,...,Standard,1189.37,38.439021,241.0,NM,40.264074,22.234178,High_spent_Medium_value_payments,403.980355,Standard
79998,21240,13193,7,Edward Krudyd,15.0,894419091.0,Manager,43782.45,3543.537500,6.0,...,Bad,3468.25,24.233841,85.0,Yes,240.664739,42.839562,Low_spent_Large_value_payments,146.741586,Poor



## Analizando las columnas

In [251]:
print(len(df_train["Payment_Behaviour"].unique()))

6


In [252]:
# Customer ID. 
# Esta se deja tal cual y no se toca, la necesitamos

# df_train['Month'].plot(kind='hist', edgecolor='black')

correlation = df_train["Annual_Income"].corr(df_train["Credit_Utilization_Ratio"])


correlation 

0.17574735571150576

Estas dos variables tienen poca correlación

In [253]:
# Age
# Edad -> Clasificar en tres grupos (va desde los 14 años hasta los 59)        juventud (14 - 26)           adultez (27 - 40)          otro (41 - 59)

# Vemos la realcion de edad con la bariable objetivosss
y = df_train["Credit_Score"]

y_numerico = OrdinalEncoder().fit_transform(df_train[["Credit_Score"]])
# Malos: Son terribles desde los 18 a los 45 años, en ese rango de valores hay un huevo de malos, Luego a partir de los 46 años baja dráticamante el número, aunque esto puede estar ligado a que haya menos datos sobre este sector de edad, tendremos que revisar eso
# Estándar: La proporcion es muy parecida a los malos, a partir de los 45 varía bastante el numero
# Bueno: En este caso se mantiene estable a partir de los 18 años hasta el final

In [254]:
# SNN
df_train.Customer_ID.nunique()
df_train.SSN.nunique()                                  # La seguridad social la vamos a borrar entonces

# Ambos devuelven tienen el mismo numero, vamos, que cada uno va ligado a un mismo cliente.

12500

## Preprocesamiento

In [255]:
df_train = df_train.drop('Month', axis=1)           # todos iguales, no aporta nada
df_train = df_train.drop('Name', axis=1)            # identificador que no aporta información
df_train = df_train.drop('SSN', axis=1)             # identificador que no aporta información
df_train = df_train.drop('Occupation', axis=1)
df_train = df_train.drop('ID', axis=1)              # identificador que no aporta información
df_train = df_train.drop('Customer_ID', axis=1)     # identificador que no aporta información
df_train = df_train.drop('Credit_Score', axis=1)    # variable objetivo
df_train = df_train.drop('Age', axis=1)
df_train = df_train.drop('Type_of_Loan', axis=1)    # hay ~6200 valores en 10000 filas, demasiado disperso

                                                    # ahora igual pero en test
df_test = df_test.drop('Month', axis=1)           
df_test = df_test.drop('Name', axis=1)            
df_test = df_test.drop('SSN', axis=1)            
df_test = df_test.drop('Occupation', axis=1)
df_test = df_test.drop('ID', axis=1)              
df_test = df_test.drop('Customer_ID', axis=1)       
df_test = df_test.drop('Age', axis=1)
df_test = df_test.drop('Type_of_Loan', axis=1)

In [256]:
df_train

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,97384.44,8167.370000,1.0,3.0,8.0,1.0,20.0,12.0,0.52,3.0,Good,405.45,32.643281,336.0,No,77.284943,53.714219,High_spent_Large_value_payments,847.172909
1,31369.58,2607.131667,8.0,9.0,27.0,8.0,30.0,25.0,19.70,6.0,Bad,4259.91,39.375507,144.0,Yes,171.748418,44.990811,High_spent_Large_value_payments,280.855383
2,30171.86,2772.321667,6.0,7.0,30.0,9.0,49.0,18.0,23.49,10.0,Bad,3406.51,34.039021,123.0,Yes,205.403926,40.781489,High_spent_Large_value_payments,247.322615
3,13100.02,1246.668333,7.0,5.0,6.0,5.0,6.0,20.0,10.64,8.0,Standard,727.29,34.747262,129.0,Yes,41.177132,23.005463,Low_spent_Large_value_payments,298.156832
4,62313.54,5120.795000,6.0,2.0,18.0,4.0,27.0,8.0,12.82,9.0,Standard,817.64,24.545394,122.0,NM,196.679642,63.346750,Low_spent_Medium_value_payments,152.669902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,107761.29,9047.107500,4.0,5.0,9.0,0.0,7.0,19.0,14.56,3.0,Standard,34.09,41.899992,211.0,Yes,0.000000,150.220682,High_spent_Large_value_payments,984.123010
79996,16218.32,1098.526667,6.0,6.0,21.0,5.0,44.0,19.0,8.98,6.0,Bad,4466.33,24.275710,98.0,NM,64.533114,34.457893,High_spent_Small_value_payments,260.523798
79997,29374.77,2377.897500,3.0,7.0,6.0,2.0,23.0,13.0,16.65,7.0,Standard,1189.37,38.439021,241.0,NM,40.264074,22.234178,High_spent_Medium_value_payments,403.980355
79998,43782.45,3543.537500,6.0,9.0,25.0,8.0,32.0,16.0,1.88,12.0,Bad,3468.25,24.233841,85.0,Yes,240.664739,42.839562,Low_spent_Large_value_payments,146.741586


In [257]:
print(len(df_train["Credit_Mix"].unique())) # Type_of_Loan: 6261 tipos, dropear
                                             # meter credit_mix 

3


In [258]:
df_train

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,97384.44,8167.370000,1.0,3.0,8.0,1.0,20.0,12.0,0.52,3.0,Good,405.45,32.643281,336.0,No,77.284943,53.714219,High_spent_Large_value_payments,847.172909
1,31369.58,2607.131667,8.0,9.0,27.0,8.0,30.0,25.0,19.70,6.0,Bad,4259.91,39.375507,144.0,Yes,171.748418,44.990811,High_spent_Large_value_payments,280.855383
2,30171.86,2772.321667,6.0,7.0,30.0,9.0,49.0,18.0,23.49,10.0,Bad,3406.51,34.039021,123.0,Yes,205.403926,40.781489,High_spent_Large_value_payments,247.322615
3,13100.02,1246.668333,7.0,5.0,6.0,5.0,6.0,20.0,10.64,8.0,Standard,727.29,34.747262,129.0,Yes,41.177132,23.005463,Low_spent_Large_value_payments,298.156832
4,62313.54,5120.795000,6.0,2.0,18.0,4.0,27.0,8.0,12.82,9.0,Standard,817.64,24.545394,122.0,NM,196.679642,63.346750,Low_spent_Medium_value_payments,152.669902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,107761.29,9047.107500,4.0,5.0,9.0,0.0,7.0,19.0,14.56,3.0,Standard,34.09,41.899992,211.0,Yes,0.000000,150.220682,High_spent_Large_value_payments,984.123010
79996,16218.32,1098.526667,6.0,6.0,21.0,5.0,44.0,19.0,8.98,6.0,Bad,4466.33,24.275710,98.0,NM,64.533114,34.457893,High_spent_Small_value_payments,260.523798
79997,29374.77,2377.897500,3.0,7.0,6.0,2.0,23.0,13.0,16.65,7.0,Standard,1189.37,38.439021,241.0,NM,40.264074,22.234178,High_spent_Medium_value_payments,403.980355
79998,43782.45,3543.537500,6.0,9.0,25.0,8.0,32.0,16.0,1.88,12.0,Bad,3468.25,24.233841,85.0,Yes,240.664739,42.839562,Low_spent_Large_value_payments,146.741586


In [259]:
oe = OrdinalEncoder()
mm = MinMaxScaler()

df_train[['Payment_of_Min_Amount']] = oe.fit_transform(df_train[['Payment_of_Min_Amount']])
df_test[['Payment_of_Min_Amount']] = oe.fit_transform(df_test[['Payment_of_Min_Amount']])
df_train[['Payment_Behaviour']] = oe.fit_transform(df_train[['Payment_Behaviour']])
df_test[['Payment_Behaviour']] = oe.fit_transform(df_test[['Payment_Behaviour']])


df_train[['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 
          'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount']] = mm.fit_transform(df_train[['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
          'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 
          'Payment_of_Min_Amount']])



df_test[['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 
          'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount']] = mm.fit_transform(df_test[['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
          'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 
          'Payment_of_Min_Amount']])
"""
column_transformer = sklearn.compose.ColumnTransformer(transformers=[
    ("Annual_Income", "passthrough" , ["Annual_Income"]),
    ("Monthly_Inhand_Salary", "passthrough" , ["Monthly_Inhand_Salary"]),
    ("Num_Bank_Accounts", "passthrough", ["Num_Bank_Accounts"]),
    ("Num_Credit_Card", "passthrough" , ["Num_Credit_Card"]),
    ("Interest_Rate","passthrough", ["Interest_Rate"]),
    ("Num_of_Loan", "passthrough" , ["Num_of_Loan"]),
    ("Delay_from_due_date", "passthrough" , ["Delay_from_due_date"]),
    ("Num_of_Delayed_Payment", "passthrough" , ["Num_of_Delayed_Payment"]),
    ("Changed_Credit_Limit", "passthrough" , ["Changed_Credit_Limit"]),
    ("Num_Credit_Inquiries","passthrough", ["Num_Credit_Inquiries"]),
    ("Outstanding_Debt","passthrough" , ["Outstanding_Debt"]),
    ("Credit_Utilization_Ratio", "passthrough" , ["Credit_Utilization_Ratio"]),
    ("Payment_of_Min_Amount", OneHotEncoder(), ["Payment_of_Min_Amount"]),
    ("Total_EMI_per_month","passthrough"  , ["Total_EMI_per_month"]),
    ("Amount_invested_monthly", "passthrough" , ["Amount_invested_monthly"]),
    ("Payment_Behaviour", OneHotEncoder(),["Payment_Behaviour"]),
    ("Monthly_Balance", "passthrough" , ["Monthly_Balance"]),
]);
"""

df_train = pd.DataFrame(df_train)
df_train

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0.522475,0.527732,0.090909,0.272727,0.212121,0.111111,0.322581,0.48,0.52,0.176471,Good,0.081079,0.421443,0.831266,0.5,77.284943,53.714219,0.0,847.172909
1,0.140846,0.154586,0.727273,0.818182,0.787879,0.888889,0.483871,1.00,19.70,0.352941,Bad,0.852304,0.645850,0.354839,1.0,171.748418,44.990811,0.0,280.855383
2,0.133922,0.165672,0.545455,0.636364,0.878788,1.000000,0.790323,0.72,23.49,0.588235,Bad,0.681550,0.467967,0.302730,1.0,205.403926,40.781489,0.0,247.322615
3,0.035230,0.063286,0.636364,0.454545,0.151515,0.555556,0.096774,0.80,10.64,0.470588,Standard,0.145475,0.491575,0.317618,1.0,41.177132,23.005463,3.0,298.156832
4,0.319732,0.323277,0.545455,0.181818,0.515152,0.444444,0.435484,0.32,12.82,0.529412,Standard,0.163553,0.151513,0.300248,0.0,196.679642,63.346750,4.0,152.669902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,0.582464,0.586771,0.363636,0.454545,0.242424,0.000000,0.112903,0.76,14.56,0.176471,Standard,0.006775,0.730000,0.521092,1.0,0.000000,150.220682,0.0,984.123010
79996,0.053257,0.053344,0.545455,0.545455,0.606061,0.555556,0.709677,0.76,8.98,0.352941,Bad,0.893606,0.142524,0.240695,0.0,64.533114,34.457893,2.0,260.523798
79997,0.129314,0.139202,0.272727,0.636364,0.151515,0.222222,0.370968,0.52,16.65,0.411765,Standard,0.237931,0.614634,0.595533,0.0,40.264074,22.234178,1.0,403.980355
79998,0.212604,0.217428,0.545455,0.818182,0.727273,0.888889,0.516129,0.64,1.88,0.705882,Bad,0.693904,0.141128,0.208437,1.0,240.664739,42.839562,3.0,146.741586


## Entrenamiento

,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0.293281,0.307993,0.818182,0.727273,0.878788,1.000000,0.983871,0.96,20.27,0.647059,Bad,0.783587,0.095125,0.387097,1.0,296.284136,53.821178,4.0,71.427393
1,0.070525,0.102587,0.454545,0.454545,0.000000,0.444444,0.016129,0.40,11.01,0.176471,Good,0.074398,0.152438,0.595533,0.5,39.417086,19.311009,5.0,211.447299
2,0.247210,0.251740,0.636364,0.454545,0.515152,0.666667,0.403226,0.36,3.30,0.647059,Standard,0.285685,0.170930,0.414392,1.0,225.693491,50.036548,5.0,198.465986
3,0.320371,0.316638,0.636364,0.727273,0.848485,0.555556,0.903226,0.80,19.58,0.352941,Standard,0.391795,0.234645,0.379653,1.0,175.360317,55.101365,0.0,472.761456
4,0.700407,0.695974,0.454545,0.636364,0.181818,0.444444,0.225806,0.00,8.79,0.294118,Good,0.298995,0.367439,0.548387,0.5,281.792301,77.036872,3.0,536.997743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.177790,0.207894,0.272727,0.363636,0.060606,0.333333,0.016129,0.40,11.60,0.176471,Good,0.164245,0.130428,0.478908,0.5,88.485663,37.562828,1.0,426.575752
19996,0.131880,0.131480,0.363636,0.363636,0.181818,0.222222,0.080645,0.04,5.99,0.235294,Good,0.140455,0.370116,0.930521,0.5,45.977299,27.931177,2.0,298.961882
19997,0.160354,0.166898,0.272727,0.181818,0.060606,0.333333,0.016129,0.48,2.48,0.176471,Good,0.043891,0.571386,0.985112,0.5,52.218894,67.741940,4.0,336.382928
19998,0.307929,0.301342,0.545455,0.636364,0.787879,0.666667,0.483871,1.00,28.85,0.647059,Bad,0.909737,0.408614,0.235732,1.0,203.605795,36.776460,1.0,381.365614


In [271]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

# 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 
#          'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount'

X = df_train[['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate',
              'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 
              'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Payment_Behaviour']]

X_test_XD = df_test[['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate',
              'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 
              'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Payment_Behaviour']]



# validacion cruzada
X_train, X_test, y_train, y_test = train_test_split(X, y_numerico, test_size=0.25, random_state=42)


xgb = XGBClassifier(
    max_depth=9,
    learning_rate=0.1,
    n_estimators=500,
    reg_alpha=1.1,
    colsample_bytree = 0.9, 
    subsample = 0.9,
    n_jobs = 5,
    objective='multi:softmax',
    num_class=3,
    random_state=42
)




#xgb.fit(X_train, y_train.ravel())       
#y_pred = xgb.predict(X_test)

xgb.fit(X, y_numerico.ravel()) 

y_pred = xgb.predict(X)                         # train
y_pred_test = xgb.predict(X_test_XD)            # test


print(f1_score(y_numerico, y_pred, average=None))               # train
print(f1_score(y_test, y_pred_test, average=None))              # test


[0.95960515 0.94807916 0.95729744]
[0.17056758 0.29400844 0.53204467]


In [261]:
submission = submission.reindex(columns=submission.columns.toList() + ["Credit_Score"])

submission["Credit_Score"] = y_pred

submission.to_csv("ML_GRUPO7.csv", index=False)


AttributeError: 'Series' object has no attribute 'columns'

## Fuentes

https://es.stackoverflow.com/questions/398654/como-ordenar-porcentajes-en-grafica-de-barras



Plot correlation in pandas

https://www.stackvidhya.com/plot-correlation-matrix-in-pandas-python/


Tablas y cosas chulas

https://relopezbriega.github.io/blog/2016/02/29/analisis-de-datos-categoricos-con-python/